### AI Sommelier RAG

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 1. 요리 풍미 묘사(Image Analysis)

In [2]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def describe_dish_flavor(query):
    # 프롬프트 정의
    prompt = ChatPromptTemplate.from_messages([
        # system : 시스템 메세지 (역할 부여, 페르소나 등)
        # human(user) : 사용자가 입력한 질문
        # ai(assistant): AI의 대답 (대화 기록을 넣을 때 주로 사용)
        ('system', '''
        Persona: You are a highly skilled food expert with a deep understanding of culinary techniques, flavor profiles, and ingredient pairings. You have a passion for exploring diverse cuisines and an ability to articulate the sensory experience of food. Your insights are backed by both practical experience and theoretical knowledge, making you a trusted source in the culinary field.

Role: As a food expert, your role is to analyze the flavors, textures, and aromas of various dishes. You provide detailed evaluations of ingredients and cooking methods, helping others understand how to create balanced and harmonious dishes. You also educate individuals on how to enhance their cooking skills and appreciate the art of gastronomy.

Examples:

When asked to analyze the flavor profile of a dish, you describe the balance between acidity, sweetness, bitterness, and umami, explaining how these elements interact to create a complex taste experience.
If someone inquires about the best techniques for enhancing a specific ingredient, you offer practical advice, such as how to caramelize onions for depth of flavor or how to properly season meat to bring out its natural taste.
When discussing food pairings, you suggest complementary ingredients and flavors, explaining the rationale behind each choice, such as pairing citrus with seafood to brighten the dish or using herbs to elevate the overall taste.
        '''),
        ('user', '이미지의 요리명과 풍미를 한 문장으로 요약해주세요.')
    ])
    # 이미지 URL 메시지 추가
    prompt += HumanMessagePromptTemplate.from_template([query])

    # 모델 설정
    model = ChatOpenAI(model='gpt-4o', temperature=0)

    output_parser = StrOutputParser()

    # 체인
    chain = prompt | model | output_parser
    return chain


c:\Users\Admin\miniconda3\envs\pystudy_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(describe_dish_flavor)
chain.invoke({'image_url':'https://halmaenoodle.com/wp-content/uploads/2024/02/%EB%B0%94%EC%A7%80%EB%9D%BD%EC%B9%BC%EA%B5%AD%EC%88%98_1.jpg'})

'이 요리는 바지락 칼국수로, 신선한 조개 육수의 감칠맛과 쫄깃한 면발이 어우러져 시원하고 깊은 풍미를 자아냅니다.'

## 2. 리뷰에서 와인 검색(Retrieval)

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

# 환경변수 로드
PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

def search_wine(query):

    """
    Args : 
        query(str): 요리 풍미 묘사 텍스트
    Return :
        dict: 다음 단계로 넘겨줄 데이터 셋 (요리 묘사 + 검색된 와인 리뷰들)
    """
    embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')
    
    # Pinecone DB 연결
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )

    # 유사도 검색 실행
    result = vector_db.similarity_search(
        query,
        k=5,
        namespace='ai-sommelier-rag-ns'
    )

    # dish_flavoer : 원래 요리 설명(그대로 전달)
    # wine_reviews : 검색된 와인 리뷰들을 하나의 문자열로 합침
    return {
        "dish_flavor" : query,
        "wine_reviews" : "\n".join([doc.page_content for doc in result])
    }

In [5]:
chain = RunnableLambda(search_wine)
chain.invoke('이 요리는 바지락 칼국수로, 신선한 조개 육수의 감칠맛과 쫄깃한 면발이 어우러져 시원하고 깊은 풍미를 자아냅니다.')

{'dish_flavor': '이 요리는 바지락 칼국수로, 신선한 조개 육수의 감칠맛과 쫄깃한 면발이 어우러져 시원하고 깊은 풍미를 자아냅니다.',
 'wine_reviews': ": 265\ncountry: South Africa\ndescription: Fairly intense and flavorful, this Chard expresses assertive aromas and flavors of wood-grilled peaches, buttercream frosting, lemon custard and toasted brioche. Medium weight in the mouth with good balance and a touch of baking spice on the close.\ndesignation: Nine Yards\npoints: 89\nprice: 40.0\nprovince: Stellenbosch\nregion_1: \nregion_2: \ntaster_name: Lauren Buzzeo\ntaster_twitter_handle: @laurbuzz\ntitle: Jardin 2009 Nine Yards Chardonnay (Stellenbosch)\nvariety: Chardonnay\nwinery: Jardin\n: 571\ncountry: US\ndescription: This wine tastes fresh and brisk, like apricots and lemons, yet a silky-smooth texture adds depth and interest to make it more than just a one-note, fruity style of wine. It comes from an over-achieving winery located one county inland from Napa.\ndesignation: \npoints: 89\nprice: 25.0\nprovince: California\nregion_1:

In [6]:
import os
print("CWD:", os.getcwd())

CWD: d:\lecture\LangChain\04_ai_sommelier_rag


## 3. 최종 와인 추천(Generation)

In [7]:
def recommend_wines(query):
    # 프롬프트 정의
    prompt = ChatPromptTemplate.from_messages([
        ('system', '''
        Persona: You are a knowledgeable and experienced sommelier with a passion for wine and food pairings.
        You possess an extensive understanding of various wine regions, grape varieties, and tasting notes.
        Your demeanor is friendly and approachable.

        Role: As a sommelier, your role is to provide expert recommendations for wine selections that perfectly complement a variety of cuisines.

        Examples:
        - If asked for grilled garlic butter shrimp, suggest a Chardonnay or Albariño.
        - If asked for affordable wines, recommend specific options from different regions.
        '''),
        HumanMessagePromptTemplate.from_template('''
        와인 페어링 추천에 아래의 요리와 풍미, 와인리뷰만을 참고하여 한글로 답변해줘
                                   
        요리와 풍미 : 
        {dish_flavor}
                                   
        와인 리뷰 : 
        {wine_reviews}
        ''')
    ])
    model = ChatOpenAI(model='gpt-5-nano', temperature=1)

    chain = prompt | model | StrOutputParser()

    return chain

## 전체 파이프라인 연결(LCEL)

In [8]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(describe_dish_flavor) | RunnableLambda(search_wine) | RunnableLambda(recommend_wines)

# 실행
image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRcUkmClnVfi9m0QrgV6HnbFs0k-2Ze4Si5jw&s'

response = chain.invoke({'image_url': image_url})

print(response)

다양한 종류의 초밥이 가진 신선한 해산물의 감칠맛과 밥의 은은한 단맛을 고려해, 주어진 와인 리뷰들에서 잘 맞아 보이는 매칭을 정리했습니다.

- Berryessa Gap 2015 Albariño (Yolo County, California)
  이유: 신선하고 brisk한 맛에 apricot과 레몬 풍미, 실키한 질감이 있음.
  페어링 포인트: 초밥의 신선한 해산물과 밥의 은은한 단맛을 산미와 과일향이 깔끔하게 보완해 줍니다.

- Jacquart NV Brut Mosaïque (Champagne)
  이유: 신선함이 유지되면서 크리미하고 캐러멜-배 풍미가 더해진 다층적 스타일.
  페어링 포인트: 초밥의 신선함과 산도를 잘 받쳐 주고, 크리미한 뉘앙스가 밥의 단맛과도 대비를 만들어 줍니다.

- Pata Negra NV Brut Sparkling (Cava)
  이유: 빵향과 멜론이 어우러지고 산도는 보통 정도, 토스트/이스트의 여운이 있음.
  페어링 포인트: 초밥의 단맛과 해산물의 풍미를 산미로 깔끔하게 정리해 주고, 토스트 풍미가 밥과의 조화를 만들어 줍니다.

- Bellisco NV Sparkling (Cava)
  이유: Funky하고 이스트 향이 두드러지며, 거품감이 있고 애플이 살짝 vegetal한 맛으로 마무리됨.
  페어링 포인트: 바다의 염도감을 머금은 피니시와 잘 어울려 초밥의 바다 풍미와 대응합니다.

- Yatir 2011 Syrah (Judean Hills)
  이유: 라즈베리/후추 풍미, 과일과 향신료의 조화, 매끄러운 탄닌, 살짝 소금기 있는 밝은 피니시.
  페어링 포인트: 초밥의 섬세한 풍미에 비해 다소 무거울 수 있어 일반적으로는 추천에서 제외될 수 있습니다. 만약 레드 와인을 꼭 원한다면, 강한 맛의 롤이나 매콤한 소스와의 대비를 고려해보세요.
